In [1]:
import numpy as np
import networkx as nx
from netrd.distance import *
import matplotlib.pyplot as plt
import datetime as dt
import itertools as it
import pandas as pd
import os

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
plt.rc('axes', axisbelow=True)

%matplotlib inline

ModuleNotFoundError: No module named 'netrd'

In [ ]:
import netrd
# dir(netrd.distance.__all__)
print(netrd.distance.__all__)

def LJ(G1, G2):
    D = LaplacianSpectral()
    d = D.dist(G1, G2, kernel='lorentzian')
    return d

def LE(G1, G2):
    D = LaplacianSpectral()
    d = D.dist(G1, G2, kernel='lorentzian', measure='euclidean')
    return d

def NJ(G1, G2):
    D = LaplacianSpectral()
    d = D.dist(G1, G2)
    return d

def NE(G1, G2):
    D = LaplacianSpectral()
    d = D.dist(G1, G2, measure='euclidean')
    return d

all_distance_function_names = ['Jaccard','Hamming','HammingIpsenMikhailov','Frobenius',
                               'PolynomialDissimilarity','DegreeDivergence','PortraitDivergence',
                               'QuantumJSD','CommunicabilityJSD','GraphDiffusion','ResistancePerturbation',
                               'NetLSD','LaplacianSpectralGJS','LaplacianSpectralEUL','IpsenMikhailov',
                               'NonBacktrackingSpectral','DistributionalNBD','DMeasure','DeltaCon','NetSimile']

all_distance_functions = [JaccardDistance().dist, Hamming().dist, HammingIpsenMikhailov().dist,
                          Frobenius().dist, PolynomialDissimilarity().dist, DegreeDivergence().dist,
                          PortraitDivergence().dist, QuantumJSD().dist, CommunicabilityJSD().dist,
                          GraphDiffusion().dist, ResistancePerturbation().dist, NetLSD().dist,
                          NJ, LE, IpsenMikhailov().dist, NonBacktrackingSpectral().dist, DistributionalNBD().dist,
                          DMeasure().dist, DeltaCon().dist, NetSimile().dist]

all_distances = dict(zip(all_distance_function_names, all_distance_functions))

better_names = ["Jaccard dissimilarity\n[JAC]",
                "Hamming distance\n[HAM]",
                "Hamming-Ipsen-Mikhailov\n[HIM]",
                "Frobenius norm\n[FRO]",
                "Polynomial dissimilarity\n[POD]",
                "Degree distribution\nJensen-Shannon div. [DJS]",
                "Portrait divergence\n[POR]",
                "Quantum density matrix\nJensen-Shannon div. [QJS]",
                "Communicability sequence\nentropy [CSE]",
                "Graph diffusion distance\n[GDD]",
                "Resistance perturbation\ndistance [REP]",
                "NetLSD\n[LSD]",
                "Laplacian (Gaussian kernel)\nJensen-Shannon div. [LGJ]",
                "Laplacian (Lorenzian kernel)\nEuclidean distance [LLE]",
                "Ipsen-Mikhailov\n[IPM]",
                "Nonbacktracking spectral\ndistance [NBD]",
                "Distribributional nonbactracking\nspectral distance [DNB]",
                "D-measure distance\n[DMD]",                
                "DeltaCon\n[DCN]",
                "NetSimile\n[NES]"
               ]
title_dict = dict(zip(all_distance_function_names,better_names))

In [ ]:
def findFilesInFolder(path, pathList, extension, subFolders = True):
    try:   # Trapping a OSError:  File permissions problem I believe
        for entry in os.scandir(path):
            if entry.is_file() and entry.path.endswith(extension):
                pathList.append(entry.path)
            elif entry.is_dir() and subFolders: 
                pathList = findFilesInFolder(entry.path, pathList, extension, subFolders)
    except OSError:
        print('Cannot access ' + path +'. Probably a permissions error')
    return pathList

def quantile999(series):
    return series.quantile(0.999)
def quantile001(series):
    return series.quantile(0.001)
def quantile99(series):
    return series.quantile(0.99)
def quantile01(series):
    return series.quantile(0.01)

# gnp and rgg

In [ ]:
dir_name_no = '../data/wegd_gnp_rgg_np.csv'
df = pd.read_csv(dir_name_no)
print(df.shape)

dir_name_no = '../cluster/out/'
extension_no = '.csv'
pathList_no = []
flist_no = findFilesInFolder(dir_name_no, pathList_no, extension_no, True)

flist_no = sorted([i for i in flist_no if 'gnp' in i or 'rgg' in i],reverse=True)
print(len(flist_no), flist_no[:2])

out = []
for fi, f in enumerate(flist_no):
    out.append(pd.read_csv(f))

dfi = pd.concat(out)
df = pd.concat([dfi,df])
df = df.replace([np.inf, -np.inf], np.nan)

n = 300
df = df.loc[df['n']==n].copy()

In [ ]:
df99 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile99}).reset_index().rename(columns={"d":"d99"})
df01 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile01}).reset_index().rename(columns={"d":"d01"})
df_n = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':'count'}).reset_index().rename(columns={"d":"d_n"})

df = df.merge(df99, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df01, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df_n, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])

df = df.loc[(df['d']>=df['d01'])&(df['d']<=df['d99'])]
print(df.shape)
df.head()

In [ ]:
means = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':np.nanmean,'d_n':np.nanmean}).reset_index()
stdvs = df.groupby(['ensemble','n','k','param_label',
                    'param_value','distance_label']).agg({'d':np.nanstd}).reset_index()
means = means.rename(columns={'d':'d_mean'})
stdvs = stdvs.rename(columns={'d':'d_stdv'})
wegdf = means.merge(stdvs, how='left', on=['ensemble','n','k','param_label', 'param_value','distance_label'])

In [ ]:
param_label = 'p'
ncols = 5
nrows = 4
w = 3.6
h = 3.0

gnp_col = 'steelblue'
gnp_col_l = 'lightskyblue'
rgg_col = 'darkred'
rgg_col_l = 'lightpink'

tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(w*ncols, h*nrows), dpi=200, sharex=True)

plt.subplots_adjust(wspace=0.275,hspace=0.325)

for ai, tup in enumerate(tups):
    d_lab = all_distance_function_names[ai]
    df_i_gnp = wegdf.loc[(wegdf['distance_label']==d_lab)&(wegdf['ensemble']=='gnp')].copy()
    df_i_rgg = wegdf.loc[(wegdf['distance_label']==d_lab)&(wegdf['ensemble']=='rgg')].copy()
    if d_lab == 'GraphDiffusion':
        wid = 4
    else:
        wid = 2

    xvals_i_gnp = df_i_gnp['param_value'].rolling(window=wid).mean().values
    means_i_gnp = df_i_gnp['d_mean'].rolling(window=wid).mean().values
    stdvs_i_gnp = df_i_gnp['d_stdv'].rolling(window=wid).mean().values
    max_i_mean_gnp = xvals_i_gnp[np.argmax(np.nan_to_num(means_i_gnp))]
    max_i_stdv_gnp = xvals_i_gnp[np.argmax(np.nan_to_num(stdvs_i_gnp))]

    ax[tup].plot(xvals_i_gnp, means_i_gnp, color=gnp_col, linewidth=2, zorder=1)
    nsamp_i_gnp = df_i_gnp['d_n'].values
    low_st = means_i_gnp-stdvs_i_gnp/(nsamp_i_gnp**0.5)
    upp_st = means_i_gnp+stdvs_i_gnp/(nsamp_i_gnp**0.5)
    ax[tup].fill_between(xvals_i_gnp, low_st, upp_st, color=gnp_col, zorder=0, alpha=0.4, linewidth=0)
    
    ax[tup].plot(xvals_i_gnp, stdvs_i_gnp, color=gnp_col, linewidth=2, zorder=1, linestyle='--')
    ax[tup].scatter([max_i_mean_gnp], [np.nanmax(means_i_gnp)], c=gnp_col_l,
                    edgecolors='.2', lw=2, s=40, zorder=2)
    ax[tup].scatter([max_i_stdv_gnp], [np.nanmax(stdvs_i_gnp)], c=gnp_col_l,
                    edgecolors='.2', lw=2, s=40, zorder=2, marker='s')

    xvals_i_rgg = df_i_rgg['param_value'].rolling(window=wid).mean().values
    means_i_rgg = df_i_rgg['d_mean'].rolling(window=wid).mean().values
    stdvs_i_rgg = df_i_rgg['d_stdv'].rolling(window=wid).mean().values
    max_i_mean_rgg = xvals_i_rgg[np.argmax(np.nan_to_num(means_i_rgg))]
    max_i_stdv_rgg = xvals_i_rgg[np.argmax(np.nan_to_num(stdvs_i_rgg))]

    ax[tup].plot(xvals_i_rgg, means_i_rgg, color=rgg_col, linewidth=2, zorder=1)
    nsamp_i_rgg = df_i_rgg['d_n'].values
    low_st = means_i_rgg-stdvs_i_rgg/(nsamp_i_rgg**0.5)
    upp_st = means_i_rgg+stdvs_i_rgg/(nsamp_i_rgg**0.5)
    ax[tup].fill_between(xvals_i_rgg, low_st, upp_st, color=rgg_col, zorder=0, alpha=0.4, linewidth=0)

    ax[tup].plot(xvals_i_rgg, stdvs_i_rgg, color=rgg_col, linewidth=2, zorder=1, linestyle='--')
    ax[tup].scatter([max_i_mean_rgg], [np.nanmax(means_i_rgg)], c=rgg_col_l,
                    edgecolors='.2', lw=2, s=40, zorder=2)
    ax[tup].scatter([max_i_stdv_rgg], [np.nanmax(stdvs_i_rgg)], c=rgg_col_l,
                    edgecolors='.2', lw=2, s=40, zorder=2, marker='s')

#     ax[tup].set_title(d_lab,fontsize='x-large')
    ax[tup].set_title(title_dict[d_lab],fontsize='large',color='.3')
    ax[tup].grid(linewidth=1, color='.6', alpha=0.4)

    ax[tup].set_yscale('log')
    ylim_i = ax[tup].get_ylim()
    ydiff = ylim_i[1] - ylim_i[0]
    ylim_i = (ylim_i[0], ylim_i[1]+0.1*ydiff)
    ax[tup].set_ylim(ylim_i[0], ylim_i[1])
    ax[tup].set_xlim(-0.025,1.025)

    if tup[0] == nrows-1:
        ax[tup].set_xlabel(r'$%s$'%param_label,fontsize='x-large')
        ax[tup].set_xticks(np.linspace(0,1,6).round(2))
        ax[tup].xaxis.set_tick_params(labelsize='large')
        if tup[1] == ncols-1:
            plotmean, = ax[tup].plot(xvals_i_rgg*100+1, means_i_rgg, color='.1',
                                     linewidth=2, zorder=0)
            maxm = ax[tup].scatter([max_i_mean_rgg], [np.nanmax(means_i_rgg)],
                                   c='w', edgecolors='.2', zorder=0, lw=2, s=40)
            plotstdv, = ax[tup].plot(xvals_i_rgg*100+1, stdvs_i_rgg, color='.4',
                                     linewidth=2, zorder=0, linestyle='--')
            maxs = ax[tup].scatter([max_i_stdv_rgg], [np.nanmax(stdvs_i_rgg)],
                                   c='w', edgecolors='.2', zorder=0, lw=2, s=40, marker='s')

            ax2 = ax[tup].twinx()
            ax2.plot(xvals_i_gnp*100, means_i_gnp, color=gnp_col, linewidth=2, zorder=0,
                     alpha=0.9, label=r'$G_{(n,p)}$')
            ax2.plot(xvals_i_rgg*100, means_i_rgg, color=rgg_col, linewidth=2, zorder=0,
                     alpha=0.9, label=r'$RGG$')
            ax2.set_yticks([])

            legend2 = ax[tup].legend([plotmean, maxm, plotstdv, maxs],
                                     ['means', 'max mean', 'standard deviations', 'max standard deviation'],
                                     bbox_to_anchor=[-1.55,-0.3], ncol=2, fontsize='large')
            ax2.legend(bbox_to_anchor=[-0.8,-0.3], ncol=1, fontsize='large')

    if tup[1] == 0:
        ax[tup].set_ylabel(r"$\langle D(G, G')_d \rangle$",fontsize='x-large')
n = 500
plt.suptitle('Within-ensemble graph distances: $G_{(n,p)}$ and $RGG$ (n=%i)'%(n),
             fontsize='xx-large', y=0.95, color='.2')

plt.savefig('../figs/pngs/%s_wegd_%s_n%i.png'%('gnp_rgg',param_label,n),bbox_inches='tight',dpi=425)
plt.savefig('../figs/pdfs/%s_wegd_%s_n%i.pdf'%('gnp_rgg',param_label,n),bbox_inches='tight')

plt.show()

___________________

# G(n,k)

In [ ]:
dir_name_no = '../cluster/out/'
extension_no = '.csv'
pathList_no = []
flist_no = findFilesInFolder(dir_name_no, pathList_no, extension_no, True)

ensemble = 'gnk'
flist_no = sorted([i for i in flist_no if ensemble in i],reverse=True)
print(len(flist_no))

out = []
for fi, f in enumerate(flist_no):
    out.append(pd.read_csv(f))

df = pd.concat(out)
df = df.replace([np.inf, -np.inf], np.nan)

In [ ]:
df99 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile99}).reset_index().rename(columns={"d":"d99"})
df01 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile01}).reset_index().rename(columns={"d":"d01"})
df_n = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':'count'}).reset_index().rename(columns={"d":"d_n"})

df = df.merge(df99, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df01, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df_n, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])

df = df.loc[(df['d']>=df['d01'])&(df['d']<=df['d99'])]
df.shape

In [ ]:
means = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':np.nanmean,'d_n':np.nanmean}).reset_index()
stdvs = df.groupby(['ensemble','n','k','param_label',
                    'param_value','distance_label']).agg({'d':np.nanstd}).reset_index()
means = means.rename(columns={'d':'d_mean'})
stdvs = stdvs.rename(columns={'d':'d_stdv'})
wegdf = means.merge(stdvs, how='left', on=['ensemble','n','k','param_label', 'param_value','distance_label'])
paramlab = wegdf['param_label'].unique()[0]
n = 500

In [ ]:
ncols = 5
nrows = 4
w = 3.6
h = 3.0

tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(w*ncols, h*nrows), dpi=200, sharex=True)

plt.subplots_adjust(wspace=0.275,hspace=0.325)
param_label = wegdf['param_label'].unique()[0]
wid = 2

for ai, tup in enumerate(tups):
    d_lab = all_distance_function_names[ai]
    df_i = wegdf.loc[(wegdf['distance_label']==d_lab)].copy()
    xvals_i = df_i.loc[df_i['n']==n]['param_value'].rolling(window=wid).mean().values
    means_i = df_i.loc[df_i['n']==n]['d_mean'].rolling(window=wid).mean().values
    stdvs_i = df_i.loc[df_i['n']==n]['d_stdv'].rolling(window=wid).mean().values
    max_i_mean = xvals_i[np.argmax(np.nan_to_num(means_i))]
    max_i_stdv = xvals_i[np.argmax(np.nan_to_num(stdvs_i))]
    
    ax[tup].plot(xvals_i, means_i, color='.1', linewidth=2, zorder=0)
    nsamp_i = df_i['d_n'].values
    low_st = means_i-stdvs_i/(nsamp_i**0.5)
    upp_st = means_i+stdvs_i/(nsamp_i**0.5)
    ax[tup].fill_between(xvals_i, low_st, upp_st, color='.5', zorder=0, alpha=0.4, linewidth=0)

    ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, zorder=0, linestyle='--')
    ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold', edgecolors='.2', lw=2, s=40)
    ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow', edgecolors='.2', lw=2, s=40, marker='s')

#     ax[tup].set_title(d_lab,fontsize='x-large')
    ax[tup].set_title(title_dict[d_lab],fontsize='large',color='.3')
    ax[tup].grid(linewidth=1, color='.6', alpha=0.4)
                 
    ax[tup].set_yscale('log')
    ax[tup].set_xscale('log')
    ylim_i = ax[tup].get_ylim()
    ydiff = ylim_i[1] - ylim_i[0]
    ylim_i = (ylim_i[0], ylim_i[1]+0.1*ydiff)
    ax[tup].set_ylim(ylim_i[0], ylim_i[1])
    ax[tup].set_xlim(9e-5, 1e3)
    ax[tup].vlines(1, ylim_i[0], ylim_i[1], color='steelblue')
    ax[tup].vlines(2, ylim_i[0], ylim_i[1], color='midnightblue')

    if tup[0] == 3:
        ax[tup].set_xlabel(r'$\langle %s \rangle$'%param_label,fontsize='x-large')
        ax[tup].xaxis.set_tick_params(labelsize='large')

        if tup[1] == ncols-1:
            v1 = ax[tup].vlines(1, ylim_i[0], ylim_i[1], color='steelblue', label=r'$\langle k \rangle = 1$')
            v2 = ax[tup].vlines(2, ylim_i[0], ylim_i[1], color='midnightblue', label=r'$\langle k \rangle = 2$')

            legend2 = ax[tup].legend([v1, v2], [r'$\langle k \rangle = 1$', r'$\langle k \rangle = 2$'],
                                     bbox_to_anchor=[-0.9,-0.3], ncol=1, fontsize='large')
            m1, = ax[tup].plot(xvals_i, means_i, color='.1', linewidth=2, zorder=0, 
                              label='means')
            maxm = ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold',
                                   edgecolors='.2', lw=2, s=40, label='max mean')
            s1, = ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, zorder=0, linestyle='--',
                              label='standard deviations')
            maxs = ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow',
                                   edgecolors='.2', lw=2, s=40, label='max standard deviation', marker='s')
            
            legend1 = ax[tup].legend([m1, maxm, s1, maxs],
                                     ['means', 'max mean', 'standard deviations', 'max standard deviation'],
                                     bbox_to_anchor=[-1.55,-0.3], ncol=2, fontsize='large')
            ax[tup].add_artist(legend2)

    if tup[1] == 0:
        ax[tup].set_ylabel(r"$\langle D(G, G')_d \rangle$",fontsize='x-large')
    
plt.suptitle(r'Within-ensemble graph distances: $G_{(n, \langle k \rangle)}$ (n=%i)'%(n),
             fontsize='xx-large', y=0.95, color='.2')

plt.savefig('../figs/pngs/%s_wegd_%s_n%i.png'%(ensemble,param_label,n),bbox_inches='tight',dpi=425)
plt.savefig('../figs/pdfs/%s_wegd_%s_n%i.pdf'%(ensemble,param_label,n),bbox_inches='tight')

plt.show()

___________________

# WS

In [ ]:
dir_name_no = '../cluster/out/'
extension_no = '.csv'
pathList_no = []
flist_no = findFilesInFolder(dir_name_no, pathList_no, extension_no, True)

ensemble = 'ws'
flist_no = sorted([i for i in flist_no if ensemble in i],reverse=True)
print(len(flist_no), flist_no[:2])

out = []
for fi, f in enumerate(flist_no):
    out.append(pd.read_csv(f))

df = pd.concat(out)
df = df.replace([np.inf, -np.inf], np.nan)

df99 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile99}).reset_index().rename(columns={"d":"d99"})
df01 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile01}).reset_index().rename(columns={"d":"d01"})
df_n = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':'count'}).reset_index().rename(columns={"d":"d_n"})

df = df.merge(df99, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df01, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df_n, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])

df = df.loc[(df['d']>=df['d01'])&(df['d']<=df['d99'])]
df.shape

In [ ]:
means = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':np.nanmean,'d_n':np.nanmean}).reset_index()
stdvs = df.groupby(['ensemble','n','k','param_label',
                    'param_value','distance_label']).agg({'d':np.nanstd}).reset_index()
means = means.rename(columns={'d':'d_mean'})
stdvs = stdvs.rename(columns={'d':'d_stdv'})
wegdf = means.merge(stdvs, how='left', on=['ensemble','n','k','param_label', 'param_value','distance_label'])

In [ ]:
ws_df = pd.read_csv('../data/watts_strogatz.csv')

In [ ]:
param_label = 'p'
ncols = 5
nrows = 4
w = 3.6
h = 3.0

tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(w*ncols, h*nrows), dpi=200, sharex=True)

plt.subplots_adjust(wspace=0.275,hspace=0.325)
paramlab = wegdf['param_label'].unique()[0]
wid = 2
n = 800
k = 8
for ai, tup in enumerate(tups):
    d_lab = all_distance_function_names[ai]
    df_i = wegdf.loc[(wegdf['distance_label']==d_lab)&(wegdf['k']==k)&(wegdf['n']==n)].copy()

    xvals_i = df_i['param_value'].rolling(window=wid).mean().values
    means_i = df_i['d_mean'].rolling(window=wid).mean().values
    stdvs_i = df_i['d_stdv'].rolling(window=wid).mean().values
    max_i_mean = xvals_i[np.argmax(np.nan_to_num(means_i))]
    max_i_stdv = xvals_i[np.argmax(np.nan_to_num(stdvs_i))]
    
    ax[tup].plot(xvals_i, means_i, color='.1', linewidth=2, zorder=0)
    nsamp_i = df_i['d_n'].values
    low_st = means_i-stdvs_i/(nsamp_i**0.5)
    upp_st = means_i+stdvs_i/(nsamp_i**0.5)
    ax[tup].fill_between(xvals_i, low_st, upp_st, color='.5', zorder=0, alpha=0.4, linewidth=0)

    ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, zorder=0, linestyle='--')
    ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold', edgecolors='.2', lw=2, s=40)
    ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow', edgecolors='.2', lw=2, s=40, marker='s')

#     ax[tup].set_title(d_lab,fontsize='x-large')
    ax[tup].set_title(title_dict[d_lab],fontsize='large',color='.3')
    ax[tup].grid(linewidth=1, color='.6', alpha=0.4)
                 
    ax[tup].set_yscale('log')
    ax[tup].set_xscale('log')
    ylim_i = ax[tup].get_ylim()
    ydiff = ylim_i[1] - ylim_i[0]
    ylim_i = (ylim_i[0], ylim_i[1]+0.1*ydiff)
    ax[tup].set_ylim(ylim_i[0], ylim_i[1])
    if tup != (nrows-1,ncols-1):
        ax2 = ax[tup].twinx()
        ax2.plot(ws_df['p'].rolling(window=wid).mean().values,
                 ws_df['path_length'].rolling(window=wid).mean().values,
                 color='steelblue', alpha=0.6)
        ax2.plot(ws_df['p'].rolling(window=wid).mean().values,
                 ws_df['clustering'].rolling(window=wid).mean().values,
                 color='midnightblue', alpha=0.6)
        ax2.set_yticks([])

    if tup[0] == nrows-1:
        ax[tup].set_xlabel(r'$%s_r$'%paramlab,fontsize='x-large')
        ax[tup].xaxis.set_tick_params(labelsize='large')
        if tup[1] == ncols-1:
            plotmean, = ax[tup].plot(xvals_i, means_i, color='.1',
                                    linewidth=2, alpha=0.95, zorder=0, label='means')
            maxm = ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold',
                                   edgecolors='.2', lw=2, s=40, label='max mean')
            plotstdv, = ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, alpha=0.95,
                                    zorder=0, linestyle='--', label='standard deviations')
            maxs = ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow',
                                   edgecolors='.2', lw=2, s=40, label='max standard deviation', marker='s')
            
            ax2 = ax[tup].twinx()
            ax2.plot(ws_df['p'].rolling(window=wid).mean().values,
                     ws_df['path_length'].rolling(window=wid).mean().values,
                     color='steelblue', alpha=0.6, label=r'$\dfrac{L_{p}}{L_{0}}$')
            ax2.plot(ws_df['p'].rolling(window=wid).mean().values,
                     ws_df['clustering'].rolling(window=wid).mean().values,
                     color='midnightblue', alpha=0.6, label=r'$\dfrac{C_{p}}{C_{0}}$')
            ax2.set_yticks([])

            legend2 = ax[tup].legend([plotmean, maxm, plotstdv, maxs],
                                     ['means', 'max mean', 'standard deviations', 'max standard deviation'],
                                     bbox_to_anchor=[-1.65,-0.3], ncol=2, fontsize='large')

            ax2.legend(bbox_to_anchor=[-0.7,-0.3], ncol=2, fontsize='large')


    if tup[1] == 0:
        ax[tup].set_ylabel(r"$\langle D(G, G')_d \rangle$",fontsize='x-large')

n = 500    
plt.suptitle('Within-ensemble graph distances: Watts-Strogatz (n=%i, k=%i)'%(n,k),
             fontsize='xx-large', y=0.95, color='.2')

plt.savefig('../figs/pngs/%s_wegd_%s_n%i_k%i.png'%(ensemble,param_label,n,k),bbox_inches='tight',dpi=425)
plt.savefig('../figs/pdfs/%s_wegd_%s_n%i_k%i.pdf'%(ensemble,param_label,n,k),bbox_inches='tight')

plt.show()

___________________

# SCM

In [ ]:
dir_name_no = '../cluster/out/'
extension_no = '.csv'
pathList_no = []
flist_no = findFilesInFolder(dir_name_no, pathList_no, extension_no, True)

ensemble = 'scm'
flist_no = sorted([i for i in flist_no if ensemble in i],reverse=True)
print(len(flist_no), flist_no[:2])

out = []
for fi, f in enumerate(flist_no):
    out.append(pd.read_csv(f))

df = pd.concat(out)
df = df.replace([np.inf, -np.inf], np.nan)

df99 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile999}).reset_index().rename(columns={"d":"d99"})
df01 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile001}).reset_index().rename(columns={"d":"d01"})
df_n = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':'count'}).reset_index().rename(columns={"d":"d_n"})

df = df.merge(df99, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df01, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df_n, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.loc[(df['d']>=df['d01'])&(df['d']<=df['d99'])]
df.shape

In [ ]:
means = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':np.nanmean,'d_n':np.nanmean}).reset_index()
stdvs = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':np.nanstd}).reset_index()

means = means.rename(columns={'d':'d_mean'})
stdvs = stdvs.rename(columns={'d':'d_stdv'})

samecols = ['ensemble', 'n', 'k', 'param_label', 'param_value', 'distance_label']
wegdf = means.merge(stdvs, how='left', on=samecols)
wegdf.head()

In [ ]:
ncols = 5
nrows = 4
w = 3.6
h = 3.0

tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(w*ncols, h*nrows), dpi=200, sharex=True)

plt.subplots_adjust(wspace=0.275,hspace=0.325)

param_label = wegdf['param_label'].unique()[0]
n = wegdf['n'].unique()[0]
k = 12

for ai, tup in enumerate(tups):
    d_lab = all_distance_function_names[ai]
    df_i = wegdf.loc[(wegdf['distance_label']==d_lab)&(wegdf['k']==k)].copy()
    if d_lab == 'GraphDiffusion':
        wid = 8
    else:
        wid = 4
    xvals_i = df_i['param_value'].rolling(window=wid).mean().values
    means_i = df_i['d_mean'].rolling(window=wid).mean().values
    stdvs_i = df_i['d_stdv'].rolling(window=wid).mean().values
    max_i_mean = xvals_i[np.argmax(np.nan_to_num(means_i))]
    max_i_stdv = xvals_i[np.argmax(np.nan_to_num(stdvs_i))]
    
    ax[tup].plot(xvals_i, means_i, color='.1', linewidth=2, alpha=0.95,
                 zorder=0)#, label='mean')

    nsamp_i = df_i['d_n'].values
    low_st = means_i-stdvs_i/(nsamp_i**0.5)
    upp_st = means_i+stdvs_i/(nsamp_i**0.5)
    ax[tup].fill_between(xvals_i, low_st, upp_st, color='.5', zorder=0, alpha=0.4, linewidth=0)

    ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold', edgecolors='.2', lw=2, s=40,
                    label='max mean\n'+r'at $\%s\approx %.2f$'%(param_label,max_i_mean))
    ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, alpha=0.95,
                 zorder=0, linestyle='--')#, label='stdv.')
    ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow', edgecolors='.2', lw=2, s=40,
                    label='max std. dev.\n'+r'at $\%s\approx %.2f$'%(param_label,max_i_stdv), marker='s')

#     ax[tup].set_title(d_lab,fontsize='x-large')
    ax[tup].set_title(title_dict[d_lab],fontsize='large',color='.3')
    ax[tup].grid(linewidth=1, color='.6', alpha=0.4)
                 
    ax[tup].set_yscale('log')
    ylim_i = ax[tup].get_ylim()
    ydiff = ylim_i[1] - ylim_i[0]
    ylim_i = (ylim_i[0], ylim_i[1]+0.1*ydiff)
    ax[tup].vlines(3, ylim_i[0], ylim_i[1], color='steelblue')
    ax[tup].set_ylim(ylim_i[0], ylim_i[1])

    if tup[0] == nrows-1:
        ax[tup].set_xlabel(r'$\%s$'%param_label,fontsize='x-large')
        ax[tup].xaxis.set_tick_params(labelsize='large')
        if tup[1] == ncols-1:
            plotmean, = ax[tup].plot(xvals_i, means_i, color='.1',
                                    linewidth=2, alpha=0.95, zorder=0, label='means')
            maxm = ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold',
                                   edgecolors='.2', lw=2, s=40, label='max mean')
            plotstdv, = ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, alpha=0.95,
                                    zorder=0, linestyle='--', label='standard deviations')
            maxs = ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow',
                                   edgecolors='.2', lw=2, s=40, label='max standard deviation', marker='s')
            
            ax2 = ax[tup].twinx()
            ax2.vlines(3, ylim_i[0], ylim_i[1], color='steelblue', label=r'$\gamma = 3$')
            ax2.set_yticks([])

            legend2 = ax[tup].legend([plotmean, maxm, plotstdv, maxs],
                                     ['means', 'max mean', 'standard deviations', 'max standard deviation'],
                                     bbox_to_anchor=[-1.5,-0.3], ncol=2, fontsize='large')

            ax2.legend(bbox_to_anchor=[-0.85,-0.3], ncol=2, fontsize='large')

    if tup[1] == 0:
        ax[tup].set_ylabel(r"$\langle D(G, G')_d \rangle$",fontsize='x-large')

    ax[tup].set_xlim(1.95,6.05)

n = wegdf.n.unique()[0]
k = wegdf.k.unique()[0]
plt.suptitle(r'Within-ensemble graph distances: soft configuration model (n=%i, k=%i)'%(n,k),
             fontsize='xx-large', y=0.95, color='.2')

plt.savefig('../figs/pngs/%s_wegd_%s_n%i_k%i.png'%(ensemble,param_label,n,k),bbox_inches='tight',dpi=425)
plt.savefig('../figs/pdfs/%s_wegd_%s_n%i_k%i.pdf'%(ensemble,param_label,n,k),bbox_inches='tight')

plt.show()

___________________

# pref attach

In [ ]:
dir_name_no = '../cluster/out/'
extension_no = '.csv'
pathList_no = []
flist_no = findFilesInFolder(dir_name_no, pathList_no, extension_no, True)

ensemble = 'pa'
flist_no = sorted([i for i in flist_no if ensemble in i],reverse=True)
print(len(flist_no))

out = []
for fi, f in enumerate(flist_no):
    out.append(pd.read_csv(f))

df = pd.concat(out)

df99 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile99}).reset_index().rename(columns={"d":"d99"})
df01 = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':quantile01}).reset_index().rename(columns={"d":"d01"})
df_n = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':'count'}).reset_index().rename(columns={"d":"d_n"})

df = df.merge(df99, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df01, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.merge(df_n, how='left', on=['ensemble','n','k','param_label','param_value','distance_label'])
df = df.loc[(df['d']>=df['d01'])&(df['d']<=df['d99'])]
df.shape

# df.head()

In [ ]:
means = df.groupby(['ensemble','n','k','param_label',
            'param_value','distance_label']).agg({'d':np.nanmean,'d_n':np.nanmean}).reset_index()
stdvs = df.groupby(['ensemble','n','k','param_label',
                    'param_value','distance_label']).agg({'d':np.nanstd}).reset_index()
means = means.rename(columns={'d':'d_mean'})
stdvs = stdvs.rename(columns={'d':'d_stdv'})
wegdf = means.merge(stdvs, how='left', on=['ensemble','n','k','param_label', 'param_value','distance_label'])
wegdf = wegdf.loc[wegdf['k']==4].copy()
paramlab = wegdf['param_label'].unique()[0]

In [ ]:
ncols = 5
nrows = 4
w = 3.6
h = 3.0

tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(w*ncols, h*nrows), dpi=200, sharex=True)

plt.subplots_adjust(wspace=0.275,hspace=0.325)
paramlab = wegdf['param_label'].unique()[0]
wid = 4
m = 1
n = 500

for ai, tup in enumerate(tups):
    d_lab = all_distance_function_names[ai]
    df_i = wegdf.loc[(wegdf['distance_label']==d_lab)].copy()
    xvals_i = df_i.loc[df_i['n']==n]['param_value'].values
    means_i = df_i.loc[df_i['n']==n]['d_mean'].values
    stdvs_i = df_i.loc[df_i['n']==n]['d_stdv'].values
    max_i_mean = xvals_i[np.argmax(np.nan_to_num(means_i))]
    max_i_stdv = xvals_i[np.argmax(np.nan_to_num(stdvs_i))]
    
    ax[tup].plot(xvals_i, means_i, color='.1', linewidth=2, zorder=0)
    nsamp_i = df_i['d_n'].values
    low_st = means_i-stdvs_i/(nsamp_i**0.5)
    upp_st = means_i+stdvs_i/(nsamp_i**0.5)
    ax[tup].fill_between(xvals_i, low_st, upp_st, color='.5', zorder=0, alpha=0.4, linewidth=0)

    ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, zorder=0, linestyle='--')
    ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold', edgecolors='.2', lw=2, s=40)
    ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow', edgecolors='.2', lw=2, s=40, marker='s')

#     ax[tup].set_title(d_lab,fontsize='x-large')
    ax[tup].set_title(title_dict[d_lab],fontsize='large',color='.3')
    ax[tup].grid(linewidth=1, color='.6', alpha=0.4)
                 
    ax[tup].set_yscale('log')
    ylim_i = ax[tup].get_ylim()
    ydiff = ylim_i[1] - ylim_i[0]
    ylim_i = (ylim_i[0], ylim_i[1]+0.1*ydiff)
    ax[tup].set_ylim(ylim_i[0], ylim_i[1])
    ax[tup].set_xlim(-5.25, 5.25)
    ax[tup].set_xticks([-4,-2,0,1,2,4])
    ax[tup].vlines(1, ylim_i[0], ylim_i[1], color='steelblue')
    ax[tup].vlines(2, ylim_i[0], ylim_i[1], color='midnightblue')

    if tup[0] == 3:
        ax[tup].set_xlabel(r'$\%s$'%paramlab,fontsize='x-large')
        ax[tup].xaxis.set_tick_params(labelsize='large')
        if tup[1] == ncols-1:
            v1 = ax[tup].vlines(1, ylim_i[0], ylim_i[1], color='steelblue', label=r'$\alpha = 1$')
            v2 = ax[tup].vlines(2, ylim_i[0], ylim_i[1], color='midnightblue', label=r'$\alpha = 2$')

            legend2 = ax[tup].legend([v1, v2], [r'$\alpha = 1$', r'$\alpha = 2$'],
                                     bbox_to_anchor=[-0.9,-0.3], ncol=1, fontsize='large')
            m1, = ax[tup].plot(xvals_i, means_i, color='.1', linewidth=2, zorder=0, 
                              label='means')
            maxm = ax[tup].scatter([max_i_mean], [np.nanmax(means_i)], c='gold',
                                   edgecolors='.2', lw=2, s=40, label='max mean')
            s1, = ax[tup].plot(xvals_i, stdvs_i, color='.4', linewidth=2, zorder=0, linestyle='--',
                              label='standard deviations')
            maxs = ax[tup].scatter([max_i_stdv], [np.nanmax(stdvs_i)], c='yellow',
                                   edgecolors='.2', lw=2, s=40, label='max standard deviation', marker='s')
            
            legend1 = ax[tup].legend([m1, maxm, s1, maxs],
                                     ['means', 'max mean', 'standard deviations', 'max standard deviation'],
                                     bbox_to_anchor=[-1.55,-0.3], ncol=2, fontsize='large')
            ax[tup].add_artist(legend2)

    if tup[1] == 0:
        ax[tup].set_ylabel(r"$\langle D(G, G')_d \rangle$",fontsize='x-large')
    
plt.suptitle('Within-ensemble graph distances: preferential attachment networks (n=%i, m=%i)'%(n,2),
             fontsize='xx-large', y=0.95, color='.2')

plt.savefig('../figs/pngs/%s_wegd_%s_n%i.png'%(ensemble,paramlab,n),bbox_inches='tight',dpi=425)
plt.savefig('../figs/pdfs/%s_wegd_%s_n%i.pdf'%(ensemble,paramlab,n),bbox_inches='tight')

plt.show()

________________

In [ ]:
dir_name_no = '../cluster/out/'
extension_no = '.csv'
pathList_no = []
flist_no = findFilesInFolder(dir_name_no, pathList_no, extension_no, True)

ensemble = 'vary_n'
flist_no = sorted([i for i in flist_no if ensemble in i],reverse=True)
print(len(flist_no))

out = []
for fi, f in enumerate(flist_no):
    out.append(pd.read_csv(f))

df = pd.concat(out)

df99 = df.groupby(['ensemble','n','param_label',
            'param_value','distance_label']).agg({'d':quantile99}).reset_index().rename(columns={"d":"d99"})
df01 = df.groupby(['ensemble','n','param_label',
            'param_value','distance_label']).agg({'d':quantile01}).reset_index().rename(columns={"d":"d01"})
df_n = df.groupby(['ensemble','n','param_label',
            'param_value','distance_label']).agg({'d':'count'}).reset_index().rename(columns={"d":"d_n"})

df = df.merge(df99, how='left', on=['ensemble','n','param_label','param_value','distance_label'])
df = df.merge(df01, how='left', on=['ensemble','n','param_label','param_value','distance_label'])
df = df.merge(df_n, how='left', on=['ensemble','n','param_label','param_value','distance_label'])
df = df.loc[(df['d']>=df['d01'])&(df['d']<=df['d99'])]
print(df.shape)

df.head()

In [ ]:
means = df.groupby(['ensemble','n','param_label',
            'param_value','distance_label']).agg({'d':np.nanmean, 'd_n':np.nanmean}).reset_index()
stdvs = df.groupby(['ensemble','n','param_label',
            'param_value','distance_label']).agg({'d':np.nanstd}).reset_index()
means = means.rename(columns={'d':'d_mean'})
stdvs = stdvs.rename(columns={'d':'d_stdv'})
wegdf = means.merge(stdvs, how='left', on=['ensemble','n','param_label', 'param_value','distance_label'])
paramlab = wegdf['param_label'].unique()[0]

In [ ]:
ncols = 5
nrows = 4
w = 3.6
h = 3.0

gnp_col = 'steelblue'
gnp_col_l = 'lightskyblue'
gnk_col = 'seagreen'
gnk_col_l = 'lightgreen'

tups = list(it.product(range(nrows), range(ncols)))
fig, ax = plt.subplots(nrows, ncols, figsize=(w*ncols, h*nrows), dpi=200, sharex=True)

plt.subplots_adjust(wspace=0.275,hspace=0.325)
paramlab = wegdf['param_label'].unique()[0]
wid = 4
m = 1

for ai, tup in enumerate(tups):
    d_lab = all_distance_function_names[ai]
    df_i_gnp = wegdf.loc[(wegdf['distance_label']==d_lab)&(wegdf['ensemble']=='gnp')].copy()
    df_i_gnk = wegdf.loc[(wegdf['distance_label']==d_lab)&(wegdf['ensemble']=='gnk')].copy()
    
    xvals_i_gnp = df_i_gnp['param_value'].values
    xvals_i_gnk = df_i_gnk['param_value'].values
    means_i_gnp = df_i_gnp['d_mean'].values
    means_i_gnk = df_i_gnk['d_mean'].values
    stdvs_i_gnp = df_i_gnp['d_stdv'].values
    stdvs_i_gnk = df_i_gnk['d_stdv'].values
    max_i_mean_gnp = xvals_i_gnp[np.argmax(np.nan_to_num(means_i_gnp))]
    max_i_mean_gnk = xvals_i_gnk[np.argmax(np.nan_to_num(means_i_gnk))]
    max_i_stdv_gnp = xvals_i_gnp[np.argmax(np.nan_to_num(stdvs_i_gnp))]
    max_i_stdv_gnk = xvals_i_gnk[np.argmax(np.nan_to_num(stdvs_i_gnk))]
    
    nsamp_i_gnp = df_i_gnp['d_n'].values
    nsamp_i_gnk = df_i_gnk['d_n'].values
    low_st_gnp = means_i_gnp-stdvs_i_gnp/(nsamp_i_gnp**0.5)
    low_st_gnk = means_i_gnk-stdvs_i_gnk/(nsamp_i_gnk**0.5)
    upp_st_gnp = means_i_gnp+stdvs_i_gnp/(nsamp_i_gnp**0.5)
    upp_st_gnk = means_i_gnk+stdvs_i_gnk/(nsamp_i_gnk**0.5)
    
    ax[tup].plot(xvals_i_gnp, means_i_gnp, color=gnp_col, linewidth=2, zorder=0)
    ax[tup].fill_between(xvals_i_gnp, low_st_gnp, upp_st_gnp, color=gnp_col, zorder=0, alpha=0.4, linewidth=0)
    ax[tup].scatter([max_i_mean_gnp], [np.nanmax(means_i_gnp)], c=gnp_col_l,
                    edgecolors='.2', lw=2, s=40)
    ax[tup].plot(xvals_i_gnp, stdvs_i_gnp, color=gnp_col, linewidth=2, zorder=0, linestyle='--')
    ax[tup].scatter([max_i_stdv_gnp], [np.nanmax(stdvs_i_gnp)], c=gnp_col_l,
                    edgecolors='.2', lw=2, s=40, marker='s')

    ax[tup].plot(xvals_i_gnk, means_i_gnk, color=gnk_col, linewidth=2, zorder=0)
    ax[tup].fill_between(xvals_i_gnk, low_st_gnk, upp_st_gnk, color=gnk_col, zorder=0, alpha=0.4, linewidth=0)
    ax[tup].scatter([max_i_mean_gnk], [np.nanmax(means_i_gnk)], c=gnk_col_l,
                    edgecolors='.2', lw=2, s=40)
    ax[tup].plot(xvals_i_gnk, stdvs_i_gnk, color=gnk_col, linewidth=2, zorder=0, linestyle='--')
    ax[tup].scatter([max_i_stdv_gnk], [np.nanmax(stdvs_i_gnk)], c=gnk_col_l,
                    edgecolors='.2', lw=2, s=40, marker='s')

    ax[tup].set_title(title_dict[d_lab],fontsize='large',color='.3')
    ax[tup].grid(linewidth=1, color='.6', alpha=0.4)

    ax[tup].set_yscale('log')
    ax[tup].set_xscale('log')
    ylim_i = ax[tup].get_ylim()
    ydiff = ylim_i[1] - ylim_i[0]
    ylim_i = (ylim_i[0], ylim_i[1]+0.1*ydiff)
    ax[tup].set_ylim(ylim_i[0], ylim_i[1])
    ax[tup].set_xlim(2.8e1,2.1e3)

    if tup[0] == nrows-1:
        ax[tup].set_xlabel(r'$%s$'%paramlab,fontsize='x-large')
        ax[tup].xaxis.set_tick_params(labelsize='large')
        if tup[1] == ncols-1:
            plotmean, = ax[tup].plot(xvals_i_gnp*100+1, means_i_gnp, color='.1',
                                     linewidth=2, zorder=0)
            maxm = ax[tup].scatter([max_i_mean_gnp], [np.nanmax(means_i_gnp)],
                                   c='w', edgecolors='.2', zorder=0, lw=2, s=40)
            plotstdv, = ax[tup].plot(xvals_i_gnp*100+1, stdvs_i_gnp, color='.4',
                                     linewidth=2, zorder=0, linestyle='--')
            maxs = ax[tup].scatter([max_i_stdv_gnp], [np.nanmax(stdvs_i_gnp)],
                                   c='w', edgecolors='.2', zorder=0, lw=2, s=40, marker='s')

            ax2 = ax[tup].twinx()
            ax2.plot(xvals_i_gnp*100, means_i_gnp, color=gnp_col, linewidth=2, zorder=0,
                     alpha=0.9, label=r'$G_{(n,p)}$')
            ax2.plot(xvals_i_gnk*100, means_i_gnk, color=gnk_col, linewidth=2, zorder=0,
                     alpha=0.9, label=r'$G_{(n,\langle k \rangle)}$')
            ax2.set_yticks([])

            legend2 = ax[tup].legend([plotmean, maxm, plotstdv, maxs],
                                     ['means', 'max mean', 'standard deviations', 'max standard deviation'],
                                     bbox_to_anchor=[-1.55,-0.3], ncol=2, fontsize='large', labelspacing=0.8)
            ax2.legend(bbox_to_anchor=[-0.8,-0.3], ncol=1, fontsize='large', labelspacing=0.35)

    if tup[1] == 0:
        ax[tup].set_ylabel(r"$\langle D(G, G')_d \rangle$",fontsize='x-large')

plt.suptitle('Within-ensemble graph distances: '+\
             r'$G_{(n,p=0.1)}$ and $G_{(n,\langle k \rangle = 6)}$, varying $n$',
             fontsize='xx-large', y=0.95, color='.2')

plt.savefig('../figs/pngs/%s_wegd_%s.png'%('gnp_gnk',paramlab),bbox_inches='tight',dpi=425)
plt.savefig('../figs/pdfs/%s_wegd_%s.pdf'%('gnp_gnk',paramlab),bbox_inches='tight')

plt.show()

In [ ]:
for nmax in [2000]:
    for _ in range(300):
        for ens in ['gnp','gnk']:
            print('python wegd-n.py %s %i 2'%(ens, nmax))